In [3]:
pip install pandas keras numpy matplotlib scikit-learn tensorflow keras-applications

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from keras.utils import to_categorical
import random
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
from glob import glob
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

/Users/mohammadshaad/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
zip1 = zipfile.ZipFile('/Users/mohammadshaad/Desktop/Everything/Early Detection of ROP/MultispectralColor.zip')
list1= zip1.namelist()
with zipfile.ZipFile('/Users/mohammadshaad/Desktop/Everything/Early Detection of ROP/MultispectralColor.zip',"r") as zip_ref:
    z1=zip_ref.extractall()

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
pip install split-folders

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import splitfolders
source_folder = '/Users/mohammadshaad/Desktop/Everything/Early Detection of ROP/MultispectralColor'


output_folder = 'split_data'


splitfolders.ratio(source_folder, output=output_folder, ratio=(0.7, 0.3))

print("Data splitting completed.")


# from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import shutil  # Importing shutil for file management


# # Assuming your image files are in /content/MultispectralColor
# data_path = "/Users/mohammadshaad/Desktop/Everything/Early Detection of ROP/MultispectralColor"

# # Get all image paths
# all_image_paths = [os.path.join(data_path, filename) for filename in os.listdir(data_path)]

# # Split data into training and testing sets (80% training, 20% testing)
# X_train, X_test = train_test_split(all_image_paths, test_size=0.2, random_state=42)

# # Create the output folder if it doesn't exist
# output_folder = "split_data"
# os.makedirs(output_folder, exist_ok=True)  # Create the folder if it doesn't exist

# # Define subfolders for training and testing data
# train_subfolder = os.path.join(output_folder, "training")
# test_subfolder = os.path.join(output_folder, "testing")
# os.makedirs(train_subfolder, exist_ok=True)  # Create subfolders if needed
# os.makedirs(test_subfolder, exist_ok=True)

# # Move files to respective folders
# for image_path in X_train:
#     shutil.move(image_path, train_subfolder)

# for image_path in X_test:
#     shutil.move(image_path, test_subfolder)

# print("Data splitting and moving completed.")

Data splitting completed.


In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.OneDeviceStrategy("/device:GPU:0")  # Use the first available GPU (T4 GPU)
else:
    strategy = tf.distribute.OneDeviceStrategy("/device:CPU:0")

In [12]:
with strategy.scope():
    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)

    train_dir = '/Users/mohammadshaad/Desktop/Everything/Early Detection of ROP/split_data/train'
    test_dir = '/Users/mohammadshaad/Desktop/Everything/Early Detection of ROP/split_data/val'

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',  # Multi-class classification
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',  # Multi-class classification
    )

Found 62 images belonging to 4 classes.
Found 29 images belonging to 4 classes.


In [13]:
# InceptionV3 Model

def InceptionV3Model():
  base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"InceptionV3 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"InceptionV3 Test Accuracy: {test_accuracy * 100:.2f}%")


In [16]:
InceptionV3Model()

87910968/87910968 [==============================] - 117s 1us/step


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_188 (Conv2D)         (None, 111, 111, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_188 (B  (None, 111, 111, 32)         96        ['conv2d_188[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_188 (Activation  (None, 111, 111, 32)         0         ['batch_normalization_188[

In [17]:
# ResNet50 Model

def ResNet50Model():
  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"ResNet50 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"ResNet50 Test Accuracy: {test_accuracy * 100:.2f}%")

In [18]:
ResNet50Model()

94765736/94765736 [==============================] - 187s 2us/step


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_4[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [19]:
# VGG16 Model

def VGG16Model():
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.Flatten()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"VGG16 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"VGG16 Test Accuracy: {test_accuracy * 100:.2f}%")



In [20]:
VGG16Model()

58889256/58889256 [==============================] - 99s 2us/step


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [21]:
# VGG19 Model

def VGG19Model():
  base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.Flatten()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"VGG19 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"VGG19 Test Accuracy: {test_accuracy * 100:.2f}%")

In [22]:
VGG19Model()

80134624/80134624 [==============================] - 38s 0us/step


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [23]:
# DenseNet201 Model

def DenseNet201Model():
  base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"DenseNet201 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"DenseNet201 Test Accuracy: {test_accuracy * 100:.2f}%")

In [24]:
DenseNet201Model()

74836368/74836368 [==============================] - 76s 1us/step


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_7[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']    

In [25]:
# MobileNet Model

def MobileNetModel():
  base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"MobileNet Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"MobileNet Test Accuracy: {test_accuracy * 100:.2f}%")


In [26]:
MobileNetModel()

17225924/17225924 [==============================] - 30s 2us/step


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128 

In [27]:
# AlexNet Model
def AlexNetModel():
  model = models.Sequential()

  model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
  model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
  model.add(layers.Conv2D(256, (5, 5), activation='relu'))
  model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))
  model.add(layers.Conv2D(384, (3, 3), activation='relu'))
  model.add(layers.Conv2D(384, (3, 3), activation='relu'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((3, 3), strides=(2, 2)))

  model.add(layers.Flatten())
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(4, activation='softmax'))  # Four classes: Stage 1, Stage 2, Stage 3, No ROP

  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()

  r = model.fit(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"AlexNet Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"AlexNet Test Accuracy: {test_accuracy * 100:.2f}%")


In [28]:
AlexNetModel()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_282 (Conv2D)         (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 26, 26, 96)        0         
 ng2D)                                                           
                                                                 
 conv2d_283 (Conv2D)         (None, 22, 22, 256)       614656    
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 10, 10, 256)       0         
 ng2D)                                                           
                                                                 
 conv2d_284 (Conv2D)         (None, 8, 8, 384)         885120    
                                                                 
 conv2d_285 (Conv2D)         (None, 6, 6, 384)         1

In [29]:
# Define the CNN model within the strategy scope
with strategy.scope():
    input_shape = (224, 224, 3)
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(4, activation='softmax')(x)  # 4 classes (No_ROP, Stage_1, Stage_2, Stage_3)

    model = Model(inputs=inputs, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the available GPU (T4 GPU)
with strategy.scope():
    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=10,
        steps_per_epoch=len(train_generator),
        validation_steps=len(test_generator)
    )

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10


2024-02-27 00:24:20.103563: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


2/2 [==============================] - 5s 3s/step - loss: 8.2174 - accuracy: 0.3548 - val_loss: 2.1478 - val_accuracy: 0.2414
Epoch 2/10
2/2 [==============================] - 4s 3s/step - loss: 2.0378 - accuracy: 0.1452 - val_loss: 1.4164 - val_accuracy: 0.2069
Epoch 3/10
2/2 [==============================] - 4s 2s/step - loss: 1.3903 - accuracy: 0.2742 - val_loss: 1.3401 - val_accuracy: 0.4138
Epoch 4/10
2/2 [==============================] - 4s 2s/step - loss: 1.3255 - accuracy: 0.4355 - val_loss: 1.3278 - val_accuracy: 0.3793
Epoch 5/10
2/2 [==============================] - 4s 2s/step - loss: 1.3412 - accuracy: 0.2742 - val_loss: 1.3309 - val_accuracy: 0.4828
Epoch 6/10
2/2 [==============================] - 4s 2s/step - loss: 1.2880 - accuracy: 0.4516 - val_loss: 1.2859 - val_accuracy: 0.4138
Epoch 7/10
2/2 [==============================] - ETA: 0s - loss: 1.2558 - accuracy: 0.4355

2024-02-27 00:24:50.573246: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


2/2 [==============================] - 4s 2s/step - loss: 1.2558 - accuracy: 0.4355 - val_loss: 1.2952 - val_accuracy: 0.4138
Epoch 8/10
2/2 [==============================] - 4s 2s/step - loss: 1.2634 - accuracy: 0.4355 - val_loss: 1.2634 - val_accuracy: 0.4138
Epoch 9/10
2/2 [==============================] - 4s 2s/step - loss: 1.2314 - accuracy: 0.4355 - val_loss: 1.2456 - val_accuracy: 0.4138
Epoch 10/10
1/1 [==============================] - 1s 709ms/step - loss: 1.2213 - accuracy: 0.4138
Test Accuracy: 41.38%
